In [41]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
# Load your full CSVs
users = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Medha Project\Data\Users.csv")
books = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Medha Project\Data\Books.csv")
ratings = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Medha Project\Data\Ratings.csv")

C:\Users\Stevi\AppData\Local\Temp\ipykernel_7248\953073624.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Medha Project\Data\Books.csv")


In [43]:
# Merge ratings with books first (using ISBN as the key)
merged_df = ratings.merge(books, on='ISBN', how='left')

# Then merge with users (using User-ID as the key)
merged_df = merged_df.merge(users, on='User-ID', how='left')

# View the result
print(f"Merged DataFrame shape: {merged_df.shape}")
merged_df.head()

Merged DataFrame shape: (1149780, 12)


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,"seattle, washington, usa",NaN
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,"h, new south wales, australia",16.0
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,"rijeka, n/a, croatia",16.0
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,"rijeka, n/a, croatia",16.0


In [44]:
df2 = merged_df.drop(columns=['Book-Title','Book-Author', 'Year-Of-Publication', 'Publisher', 'Location', 'Age','Image-URL-S', 'Image-URL-M', 'Image-URL-L'])

In [45]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [46]:
# Create mappings for users and books
user_ids = df2['User-ID'].unique()
book_ids = df2['ISBN'].unique()

user_map = {user_id: idx for idx, user_id in enumerate(user_ids)}
book_map = {book_id: idx for idx, book_id in enumerate(book_ids)}

# Map to indices
df2['user_idx'] = df2['User-ID'].map(user_map)
df2['book_idx'] = df2['ISBN'].map(book_map)

# Create sparse matrix
num_users = len(user_ids)
num_books = len(book_ids)

In [47]:
user_item_sparse = csr_matrix(
    (df2['Book-Rating'].values, (df2['user_idx'].values, df2['book_idx'].values)),
    shape=(num_users, num_books)
)

print(f"Sparse matrix shape: {user_item_sparse.shape}")
print(f"Matrix density: {user_item_sparse.nnz / (num_users * num_books):.4%}")

Sparse matrix shape: (105283, 340556)
Matrix density: 0.0032%


In [48]:
# Calculate user similarity (this may still be slow for very large datasets)
# Consider sampling or using approximate methods
user_similarity_sparse = cosine_similarity(user_item_sparse, dense_output=False)

# Function to get recommendations using sparse matrix
def get_recommendations_sparse(user_id, top_n=10, n_similar_users=20):
    if user_id not in user_map:
        return f"User {user_id} not found"
    
    user_idx = user_map[user_id]
    
    # Get similarity scores for this user
    user_sim_scores = user_similarity_sparse[user_idx].toarray().flatten()
    
    # Get top similar users (excluding self)
    similar_user_indices = np.argsort(user_sim_scores)[::-1][1:n_similar_users+1]
    similar_scores = user_sim_scores[similar_user_indices]
    
    # Get weighted ratings from similar users
    similar_users_ratings = user_item_sparse[similar_user_indices]
    weighted_sum = similar_users_ratings.T.dot(similar_scores)
    
    # Convert to array if it's a matrix
    if hasattr(weighted_sum, 'A1'):
        weighted_ratings = (weighted_sum / similar_scores.sum()).A1
    else:
        weighted_ratings = (weighted_sum / similar_scores.sum()).flatten()
    
    # Remove books already rated by the user
    user_rated = user_item_sparse[user_idx].nonzero()[1]
    weighted_ratings[user_rated] = -1
    
    # Get top recommendations
    top_book_indices = np.argsort(weighted_ratings)[::-1][:top_n]
    top_scores = weighted_ratings[top_book_indices]
    
    # Map back to ISBNs
    reverse_book_map = {idx: book_id for book_id, idx in book_map.items()}
    recommended_isbns = [reverse_book_map[idx] for idx in top_book_indices]
    
    return pd.DataFrame({
        'ISBN': recommended_isbns,
        'Score': top_scores
    })

In [49]:
# Test the function
sample_user = df2['User-ID'].iloc[100]
recommendations = get_recommendations_sparse(sample_user, top_n=10)
print(f"\nRecommendations for User {sample_user}:")
print(recommendations)


Recommendations for User 276800:
          ISBN  Score
0   0385086954   10.0
1  08883961962   10.0
2   0385182449   10.0
3   0452279178   10.0
4   0385318294   10.0
5   0446525502   10.0
6   0446531332   10.0
7   0440202477   10.0
8   0786890436   10.0
9   0670272663   10.0


In [51]:
# Get book details
recommended_books = books[books['ISBN'].isin(recommendations['ISBN'])][['ISBN', 'Book-Title', 'Book-Author']]
result = recommendations.merge(recommended_books, on='ISBN')
print("\nWith book details:")
display(result)


With book details:


,ISBN,Score,Book-Title,Book-Author
0,0385086954,10.0,Carrie,Stephen King
1,0385182449,10.0,Pet Sematary,Stephen King
2,0452279178,10.0,"Wizard and Glass (The Dark Tower, Book 4)",Stephen King
3,0385318294,10.0,A Dry Spell,Susie Moloney
4,0446525502,10.0,The Rescue,Nicholas Sparks
5,0446531332,10.0,Nights in Rodanthe,Nicholas Sparks
6,0440202477,10.0,Sleeping Tiger,ROSAMUNDE PILCHER
7,0786890436,10.0,The Diary of Ellen Rimbauer: My Life at Rose Red,Ellen Rimbauer
8,0670272663,10.0,Different Seasons,Stephen King
